# Udacity AI - Project 1: Sudoku

*Cell0 is from my template*

In [1]:
import sys
from pathlib import Path

print('Python ver: {}\nPython env: {}'.format(sys.version, Path(sys.prefix).name))
print('Currrent dir: {}\n'.format(Path.cwd()))

def add_to_sys_path(this_path, up=False):
    """
    Prepend this_path to sys.path.
    If up=True, path refers to parent folder (1 level up).
    """
    if up:
        # NB: Path does not have a str method.
        newp = Path(this_path).parent.as_posix()
    else:
        newp = Path(this_path).as_posix()
    
    if newp not in sys.path:
        sys.path.insert(1, newp)
        print('Path added to sys.path: {}'.format(newp))

# if notebook inside another folder, eg ./notebooks:
nb_folder = 'notebooks'
add_to_sys_path(Path.cwd(), Path.cwd().name.startswith(nb_folder))


def get_project_dirs(which=['data', 'images'], nb_folder='notebooks'):
    dir_lst = []
    if Path.cwd().name.startswith(nb_folder):
        dir_fn = Path.cwd().parent.joinpath
    else:
        dir_fn = Path.cwd().joinpath
        
    for d in which:
        DIR = dir_fn(d)
        if not DIR.exists():
            Path.mkdir(DIR)
        dir_lst.append(DIR)
    return dir_lst

DIR_DATA, DIR_IMG = get_project_dirs()
    
import numpy as np
import scipy as sp
from scipy import stats as sps
import pandas as pd
#pd.set_option("display.max_colwidth", 200)

import matplotlib as mpl
from matplotlib import pyplot as plt
plt.ion()
plt.style.use('seaborn-muted')

from pprint import pprint as pp

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display, Markdown #, HTML
##display(HTML("<style>.container { width:100% !important; }</style>"))
#def new_section(title):
#    style = "text-align:center;background:#c2d3ef;padding:20px;color:#ffffff;font-size:3em;width:98%"
#    return HTML('<div style="{}">{}</div>'.format(style, title))


# Filtered dir() for method discovery:
def filter_module_dir(mdl, filter_str=None, start_with_str='_', exclude=True):
    """Filter dir(mdl) for method discovery.
       Input:
       :param mdl (object): module, optionally with submodule path(s), e.g. mdl.submdl1.submdl2.
       :param filter_str (str, None): filter all method names containing that string.
       :param start_with_str (str, '_'), exclude (bool, True): start_with_str and exclude work together
              to perform search on non-dunder methods (default).
    """
    search_dir = [d for d in dir(mdl) if not d.startswith(start_with_str) == exclude]
    if filter_str is None:
        return search_dir
    else:
        filter_str = filter_str.lower()
        return [d for d in search_dir if d.lower().find(filter_str) != -1]


def get_mdl_pkgs(alib):
    import inspect
    "Inspect module hierarchy down two levels only."
    for name, mdl in inspect.getmembers(alib, inspect.ismodule):
        print('\n{:>13} : {}'.format(mdl.__name__, filter_dir(mdl)))
        for mdl_name, mdl_sub in inspect.getmembers(mdl, inspect.ismodule):
            if mdl_sub.__doc__:
                print('\n{:>20} : {}'.format(mdl_name, mdl_sub.__doc__.strip()))

# these no longer work in JupyterLab
%load_ext autoreload
%autoreload 2 

Python ver: 3.6.7 (default, Feb 28 2019, 07:28:18) [MSC v.1900 64 bit (AMD64)]
Python env: dsml
Currrent dir: C:\Users\catch\Documents\GitHub\AI\Projects\1_Sudoku\notebooks

Path added to sys.path: C:/Users/catch/Documents/GitHub/AI/Projects/1_Sudoku


In [2]:
from collections import defaultdict
import solution as sol

In [3]:
rows = sol.rows
cols = sol.cols

boxes = sol.boxes
unitlist = sol.unitlist
row_units = sol.row_units
column_units = sol.column_units
square_units = sol.square_units
diag_units_sol = sol.diag_units

units = sol.units
peers = sol.peers

### During troubleshooting, my mentor suggested that I changed this expression:
```
pool = peers[boxA].intersection(peers[peerA])
```
to:
```
pool = set(peers[boxA].intersection(peers[peerA]))
```
**My claim**: this would not have changed anything, because it is unncessesary.  
**Proof to show that wrapping the expression with set() in pool2 is unnecessary as peers[box] returns a set.**

In [4]:
boxA = 'A1'
peerA = 'C4'
P = {'A1':{'A2','A3','B1', 'B2', 'B3', 'C1', 'C3'},
     'C4':{'C1','C2', 'C3', 'D1', 'D4', 'B3', 'E5','I9'}}

pool1 = P[boxA].intersection(P[peerA])
pool2 = set(P[boxA].intersection(P[peerA]))
print(pool1 == pool2)

True


# I used this helper function to verify the number of peers for the Sudoku with added diagonal constraint:

In [5]:
peers_len_d = defaultdict(list)
for k, v in peers.items():
    peers_len_d[len(v)].append(k)
    
for k in peers_len_d.keys():
    print(f'{k}:\n\t', peers_len_d[k])

26:
	 ['A1', 'A9', 'B2', 'B8', 'C3', 'C7', 'D4', 'D6', 'F4', 'F6', 'G3', 'G7', 'H2', 'H8', 'I1', 'I9']
20:
	 ['A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'B1', 'B3', 'B4', 'B5', 'B6', 'B7', 'B9', 'C1', 'C2', 'C4', 'C5', 'C6', 'C8', 'C9', 'D1', 'D2', 'D3', 'D5', 'D7', 'D8', 'D9', 'E1', 'E2', 'E3', 'E4', 'E6', 'E7', 'E8', 'E9', 'F1', 'F2', 'F3', 'F5', 'F7', 'F8', 'F9', 'G1', 'G2', 'G4', 'G5', 'G6', 'G8', 'G9', 'H1', 'H3', 'H4', 'H5', 'H6', 'H7', 'H9', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8']
32:
	 ['E5']


> The naked twins strategy says that if you have two or more unallocated boxes 
in a unit and there are only two digits that can go in those two boxes, then 
those two digits can be eliminated from the possible assignments of all other 
boxes in the same unit.

# Individual test values checks:

In [6]:
test_grid0 = '..3.2.6..9003.5..1..18.64....81.29..7......8..67.82....26.95..8..2.3..9..5.1.3.0'
test_grid1 = '..3.2.6..9003.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3.0'
test_grid2 = '4.....8.5.3.....0....7......2.....6...c.8.4......1.......6.3.7.5..2.....1.4......'

diagonal_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
solved_diag_sudoku = {'G7': '8', 'G6': '9', 'G5': '7', 'G4': '3', 'G3': '2', 'G2': '4', 'G1': '6', 'G9': '5',
                          'G8': '1', 'C9': '6', 'C8': '7', 'C3': '1', 'C2': '9', 'C1': '4', 'C7': '5', 'C6': '3',
                          'C5': '2', 'C4': '8', 'E5': '9', 'E4': '1', 'F1': '1', 'F2': '2', 'F3': '9', 'F4': '6',
                          'F5': '5', 'F6': '7', 'F7': '4', 'F8': '3', 'F9': '8', 'B4': '7', 'B5': '1', 'B6': '6',
                          'B7': '2', 'B1': '8', 'B2': '5', 'B3': '3', 'B8': '4', 'B9': '9', 'I9': '3', 'I8': '2',
                          'I1': '7', 'I3': '8', 'I2': '1', 'I5': '6', 'I4': '5', 'I7': '9', 'I6': '4', 'A1': '2',
                          'A3': '7', 'A2': '6', 'E9': '7', 'A4': '9', 'A7': '3', 'A6': '5', 'A9': '1', 'A8': '8',
                          'E7': '6', 'E6': '2', 'E1': '3', 'E3': '4', 'E2': '8', 'E8': '5', 'A5': '4', 'H8': '6',
                          'H9': '4', 'H2': '3', 'H3': '5', 'H1': '9', 'H6': '1', 'H7': '7', 'H4': '2', 'H5': '8',
                          'D8': '9', 'D9': '2', 'D6': '8', 'D7': '1', 'D4': '4', 'D5': '3', 'D2': '7', 'D3': '6',
                          'D1': '5'}

before_naked_twins_1 = {'I6': '4', 'H9': '3', 'I2': '6', 'E8': '1', 'H3': '5', 'H7': '8', 'I7': '1', 'I4': '8',
                            'H5': '6', 'F9': '7', 'G7': '6', 'G6': '3', 'G5': '2', 'E1': '8', 'G3': '1', 'G2': '8',
                            'G1': '7', 'I1': '23', 'C8': '5', 'I3': '23', 'E5': '347', 'I5': '5', 'C9': '1', 'G9': '5',
                            'G8': '4', 'A1': '1', 'A3': '4', 'A2': '237', 'A5': '9', 'A4': '2357', 'A7': '27',
                            'A6': '257', 'C3': '8', 'C2': '237', 'C1': '23', 'E6': '579', 'C7': '9', 'C6': '6',
                            'C5': '37', 'C4': '4', 'I9': '9', 'D8': '8', 'I8': '7', 'E4': '6', 'D9': '6', 'H8': '2',
                            'F6': '125', 'A9': '8', 'G4': '9', 'A8': '6', 'E7': '345', 'E3': '379', 'F1': '6',
                            'F2': '4', 'F3': '23', 'F4': '1235', 'F5': '8', 'E2': '37', 'F7': '35', 'F8': '9',
                            'D2': '1', 'H1': '4', 'H6': '17', 'H2': '9', 'H4': '17', 'D3': '2379', 'B4': '27',
                            'B5': '1', 'B6': '8', 'B7': '27', 'E9': '2', 'B1': '9', 'B2': '5', 'B3': '6', 'D6': '279',
                            'D7': '34', 'D4': '237', 'D5': '347', 'B8': '3', 'B9': '4', 'D1': '5'}
possible_solutions_1 = [
        {'G7': '6', 'G6': '3', 'G5': '2', 'G4': '9', 'G3': '1', 'G2': '8', 'G1': '7', 'G9': '5', 'G8': '4', 'C9': '1',
         'C8': '5', 'C3': '8', 'C2': '237', 'C1': '23', 'C7': '9', 'C6': '6', 'C5': '37', 'A4': '2357', 'A9': '8',
         'A8': '6', 'F1': '6', 'F2': '4', 'F3': '23', 'F4': '1235', 'F5': '8', 'F6': '125', 'F7': '35', 'F8': '9',
         'F9': '7', 'B4': '27', 'B5': '1', 'B6': '8', 'B7': '27', 'E9': '2', 'B1': '9', 'B2': '5', 'B3': '6', 'C4': '4',
         'B8': '3', 'B9': '4', 'I9': '9', 'I8': '7', 'I1': '23', 'I3': '23', 'I2': '6', 'I5': '5', 'I4': '8', 'I7': '1',
         'I6': '4', 'A1': '1', 'A3': '4', 'A2': '237', 'A5': '9', 'E8': '1', 'A7': '27', 'A6': '257', 'E5': '347',
         'E4': '6', 'E7': '345', 'E6': '579', 'E1': '8', 'E3': '79', 'E2': '37', 'H8': '2', 'H9': '3', 'H2': '9',
         'H3': '5', 'H1': '4', 'H6': '17', 'H7': '8', 'H4': '17', 'H5': '6', 'D8': '8', 'D9': '6', 'D6': '279',
         'D7': '34', 'D4': '237', 'D5': '347', 'D2': '1', 'D3': '79', 'D1': '5'},
        {'I6': '4', 'H9': '3', 'I2': '6', 'E8': '1', 'H3': '5', 'H7': '8', 'I7': '1', 'I4': '8', 'H5': '6', 'F9': '7',
         'G7': '6', 'G6': '3', 'G5': '2', 'E1': '8', 'G3': '1', 'G2': '8', 'G1': '7', 'I1': '23', 'C8': '5', 'I3': '23',
         'E5': '347', 'I5': '5', 'C9': '1', 'G9': '5', 'G8': '4', 'A1': '1', 'A3': '4', 'A2': '237', 'A5': '9',
         'A4': '2357', 'A7': '27', 'A6': '257', 'C3': '8', 'C2': '237', 'C1': '23', 'E6': '579', 'C7': '9', 'C6': '6',
         'C5': '37', 'C4': '4', 'I9': '9', 'D8': '8', 'I8': '7', 'E4': '6', 'D9': '6', 'H8': '2', 'F6': '125',
         'A9': '8', 'G4': '9', 'A8': '6', 'E7': '345', 'E3': '79', 'F1': '6', 'F2': '4', 'F3': '23', 'F4': '1235',
         'F5': '8', 'E2': '3', 'F7': '35', 'F8': '9', 'D2': '1', 'H1': '4', 'H6': '17', 'H2': '9', 'H4': '17',
         'D3': '79', 'B4': '27', 'B5': '1', 'B6': '8', 'B7': '27', 'E9': '2', 'B1': '9', 'B2': '5', 'B3': '6',
         'D6': '279', 'D7': '34', 'D4': '237', 'D5': '347', 'B8': '3', 'B9': '4', 'D1': '5'}
        ]
    

In [7]:
result = sol.solve(diagonal_grid)
sol.display(result)

2 6 7 |9 4 5 |3 8 1 
8 5 3 |7 1 6 |2 4 9 
4 9 1 |8 2 3 |5 7 6 
------+------+------
5 7 6 |4 3 8 |1 9 2 
3 8 4 |1 9 2 |6 5 7 
1 2 9 |6 5 7 |4 3 8 
------+------+------
6 4 2 |3 7 9 |8 1 5 
9 3 5 |2 8 1 |7 6 4 
7 1 8 |5 6 4 |9 2 3 


In [8]:
solved = sol.values2grid(solved_diag_sudoku)
result = sol.solve(solved)
sol.display(result)

2 6 7 |9 4 5 |3 8 1 
8 5 3 |7 1 6 |2 4 9 
4 9 1 |8 2 3 |5 7 6 
------+------+------
5 7 6 |4 3 8 |1 9 2 
3 8 4 |1 9 2 |6 5 7 
1 2 9 |6 5 7 |4 3 8 
------+------+------
6 4 2 |3 7 9 |8 1 5 
9 3 5 |2 8 1 |7 6 4 
7 1 8 |5 6 4 |9 2 3 


In [9]:
sol.display(before_naked_twins_1)

  1   237   4  | 2357  9   257 |  27   6    8  
  9    5    6  |  27   1    8  |  27   3    4  
  23  237   8  |  4    37   6  |  9    5    1  
---------------+---------------+---------------
  5    1   2379| 237  347  279 |  34   8    6  
  8    37  379 |  6   347  579 | 345   1    2  
  6    4    23 | 1235  8   125 |  35   9    7  
---------------+---------------+---------------
  7    8    1  |  9    2    3  |  6    4    5  
  4    9    5  |  17   6    17 |  8    2    3  
  23   6    23 |  8    5    4  |  1    7    9  


In [10]:
sol.display(sol.naked_twins(before_naked_twins_1))

  1   237   4  | 2357  9   257 |  27   6    8  
  9    5    6  |  27   1    8  |  27   3    4  
  23  237   8  |  4    37   6  |  9    5    1  
---------------+---------------+---------------
  5    1    79 | 237  347  279 |  34   8    6  
  8    37   79 |  6   347  579 | 345   1    2  
  6    4    23 | 1235  8   125 |  35   9    7  
---------------+---------------+---------------
  7    8    1  |  9    2    3  |  6    4    5  
  4    9    5  |  17   6    17 |  8    2    3  
  23   6    23 |  8    5    4  |  1    7    9  
